In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U loralib
!pip install -q -U einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.4 MB/s eta 0:00:00


## CONSTANT

In [2]:
MODEL_NAME = "vilm/vinallama-7b-chat" # model name
FINETUNED_MODEL = "finetuned_model"

PROMTP_FORMAT = """
<|im_start|>system
{}
<|im_end|>
<|im_start|>user
{}
<|im_end|>
<|im_start|>assistant
{}
"""


IN_CONTEXT_PROMPT = {
    "2": "Bạn là một chuyên gia về toán học. Trả lời câu hỏi sau bằng cách đưa ra đáp án chính xác nhất. Đáp án sẽ là một trong các lựa chọn A, B, C, D. Hãy suy nghĩ từng bước một."
    ,"1": "Bạn là một chuyên gia về toán học. Trả lời câu hỏi sau bằng cách đưa ra đáp án chính xác nhất. Hãy suy nghĩ từng bước một."
}

DATASETS = [
    "hllj/vi_gsm8k",
    "hllj/vi_grade_school_math_mcq"
] # list of datasets used for training

BIT_QUANTIZATION = "4bit" # model quantization - can change to 8bit

# MODEL GENERATE CONFIG
MAX_NEW_TOKENS=200 # max length of generated tokens
TEMPERATURE=0.7 # controls randomness in generation
TOP_P=0.7 # nucleus sampling parameter
NUM_RETURN_SEQUENCES=1 # number of generated sequences to return

# DIRECTORY

DATA_PATH = "data/"
TRAINING_TOKENIZED_DATASET = "data/training_tokenized_dataset"
TRAINING_DATASET = "data/training_dataset"


## MODEL CONFIG

In [3]:
import os
import torch
# from constant import (
#     MODEL_NAME, R, LORA_ALPHA, TARGET_MODULES, LORA_DROPOUT, BIAS, TASK_TYPE, 
#     MAX_NEW_TOKENS, TEMPERATURE, TOP_P, NUM_RETURN_SEQUENCES
# )
from transformers import (
    AutoConfig, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
)
from peft import (
    LoraConfig, PeftConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "up_proj",
        "o_proj",
        "k_proj",
        "down_proj",
        "gate_proj",
        "v_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
def get_tokenizer():
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer

def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
    )

def get_generate_config(tokenizer, model):
    generation_config = model.generation_config
    generation_config.max_new_tokens = MAX_NEW_TOKENS
    generation_config.temperature = TEMPERATURE
    generation_config.top_p = TOP_P
    generation_config.num_return_sequences = NUM_RETURN_SEQUENCES
    generation_config.pad_token_id = tokenizer.eos_token_id
    generation_config.eos_token_id = tokenizer.eos_token_id

    return generation_config

## PREPARE DATASET

In [4]:
from datasets import load_dataset, Dataset
# from constant import (
#     DATASETS, MODEL_NAME, PROMTP_FORMAT, IN_CONTEXT_PROMPT, TRAINING_TOKENIZED_DATASET, TRAINING_DATASET)
# from config_model import get_tokenizer
from tqdm import tqdm
import random

# ds1_format: 
# {
#     "index": 0,
#     "question": "Natalia đã bán kẹp tóc cho 48 người bạn của cô ấy vào tháng 4, và sau đó cô ấy đã bán nửa số lượng kẹp tóc đó vào tháng 5. Natalia đã bán tổng cộng bao nhiêu kẹp tóc trong tháng 4 và tháng 5?",
#     "explanation": "Natalia đã bán 24 kẹp trong tháng 5.\nNatalia đã bán tổng cộng 72 kẹp trong tháng 4 và tháng 5.",
#     "answer": "72"
# }

# ds2_format: note that "problems"
# {
#   "id": "f9decb7530da8097ebca80315928825e",
#   "question": "Câu 2: Trang 21 - sgk toán lớp 5\nMột gia đình gồm 3 người (bố, mẹ và một con). Bình quân thu nhập hàng tháng 800 000 đồng mỗi người. Nếu gia đình đó có thêm một con nữa mà tổng thu nhập của gia đình không thay đổi thì bình quân thu nhập hàng tháng của mỗi người giảm đi bao nhiêu tiền?",
#   "explanation": "Tổng thu hập bình quân một tháng của gia đình đó là:\n800000 x 3 = 2400000 ( đồng)\nSau khi thêm một người, thu nhập trung bình của một người trong gia đình là:\n2400000 : 4 = 600000 ( đồng)\nVậy so với trước đó, thu nhập bình quân mỗi tháng của một người đã giảm đi:\n800000 - 600000 = 200000 ( đồng)\nĐáp án: 200000 đồng.",
#   "choices": [
#       "A. 180000 đồng.",
#       "B. 250000 đồng.",
#       "C. 220000 đồng.",
#       "D. 200000 đồng."
#   ],
#   "answer": "D. 200000 đồng."
# }

def generate_prompt_tokenize(num_ds: int, tokenizer, question: str, explanation: str, choices: str = None, return_tokenized_sample:bool = False):
    in_context_prompt = IN_CONTEXT_PROMPT[str(num_ds)]
    if num_ds == 1:
        instruction_prompt = f"""
        ### Câu hỏi:
        {question}
        """.strip()
    else:
        instruction_prompt = f"""
        ### Câu hỏi:
        {question}
        ### Các lựa chọn:
        {choices}
        """.strip()
    full_prompt = PROMTP_FORMAT.format(
        in_context_prompt,
        instruction_prompt,
        explanation
    )
    if return_tokenized_sample:
        full_prompt = tokenizer(
            full_prompt,
            padding=True,
            truncation=True,
            max_length=512
        )
    else:
        full_prompt = {
            "prompt": full_prompt
        }
    return full_prompt

def prepare_ds1(tokenizer, training_samples:list, return_tokenized_sample:bool = False):
    print("Preparing Dataset 1...")
    ds1 = load_dataset(DATASETS[0]) # has train and test
    for sample in tqdm(ds1["train"]):
        question = sample["question"]
        explanation = sample["explanation"]

        if explanation == '' or question == '':
            continue
        training_sample = generate_prompt_tokenize(1, tokenizer, question, explanation, return_tokenized_sample=return_tokenized_sample)
        training_samples.append(training_sample)

def prepare_ds2(tokenizer, training_samples:list, return_tokenized_sample:bool = False):
    print("Preparing Dataset 2...")
    ds2 = load_dataset(DATASETS[1]) # has only train
    for sample in tqdm(ds2["train"]):
        for quest in sample['problems']:
            choices = quest["choices"]
            question = quest["question"]
            explanation = quest["explanation"]
            
            if explanation == '' or question == '' or choices == []:
                continue
            try:
                question = question.split("\n \n")[1].strip()
            except:
                continue

            choices = "\n".join(choices)
            training_sample = generate_prompt_tokenize(2, tokenizer, question, explanation, choices, return_tokenized_sample=return_tokenized_sample)
            training_samples.append(training_sample)
    
def prepare_dataset(return_tokenized_sample=True, is_save=False):
    print("Get tokenizer...")
    tokenizer = get_tokenizer()
    training_samples = []
    prepare_ds1(tokenizer, training_samples, return_tokenized_sample)
    prepare_ds2(tokenizer, training_samples, return_tokenized_sample)

    # Shuffling training samples
    random.shuffle(training_samples)

    # Save dataset
    training_dataset = Dataset.from_list(training_samples)
    if is_save:
        if return_tokenized_sample:
            training_dataset.save_to_disk(TRAINING_TOKENIZED_DATASET) # tokenized
        else:
            training_dataset.save_to_disk(TRAINING_DATASET) # prompt
    
    return training_dataset


## TRAIN

In [5]:
import os
import torch

os.environ["WANDB_DISABLED"] = "true"  # Disable wandb

from datasets import load_from_disk
from transformers import (
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
from peft import (
    get_peft_model,
    prepare_model_for_kbit_training
)

# Load Training Dataset
if not os.path.exists(TRAINING_TOKENIZED_DATASET):
    print(f"Training dataset not found at {TRAINING_DATASET}")
    training_dataset = prepare_dataset(True, True)
else:
    training_dataset = load_from_disk(TRAINING_TOKENIZED_DATASET)
print("Training Dataset is Loaded")

# Get Tokenizer
tokenizer = get_tokenizer()

# Load Model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)


print("Trainable Params after LoRA config:")
print_trainable_parameters(model)

generation_config = get_generate_config(tokenizer, model)

training_args = TrainingArguments(
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=2,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=10,
    output_dir="model/experiments",
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    dataloader_num_workers=4,
    report_to="none",
    ddp_find_unused_parameters=False,  # Hỗ trợ multi-GPU
)

trainer = Trainer(
    model=model,
    train_dataset=training_dataset,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False

trainer.train()


Training dataset not found at data/training_dataset
Get tokenizer...


tokenizer_config.json:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Preparing Dataset 1...


...

Trainable Params after LoRA config:
trainable params: 39976960 || all params: 3657576448 || trainables%: 1.092990415056391


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.078100
20,2.004900
30,1.187200
40,1.000200
50,0.957400
60,0.901800
70,0.864100
80,0.835300
90,0.832700
100,0.818500


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=1040, training_loss=0.7348343821672293, metrics={'train_runtime': 33807.9601, 'train_samples_per_second': 0.985, 'train_steps_per_second': 0.031, 'total_flos': 3.9534653443640525e+17, 'train_loss': 0.7348343821672293, 'epoch': 1.9971181556195965})

In [6]:
# Save the final model and tokenizer
model.save_pretrained("final_model")
tokenizer.save_pretrained("final_model")

('final_model/tokenizer_config.json',
 'final_model/special_tokens_map.json',
 'final_model/tokenizer.json')